In [1]:
import os
import os
import errno

In [2]:
# Habría que hacer un mapeo de este tipo para los tags de cada categoría
# "V" debe coincidir con la posición 0 del tag que es el nombre de la categoría
# Hay que especificar la posición en el tag de cada rasgo y si hay algo para mapear, también.
# Si no hay nada para mapear con el valor que tiene en el tag en esa posición, deja el valor del tag.

tags_mapping={
    "V":{
        "subcategoria":{
            "posicion": 1
        },
        "modo": {
            "posicion": 2,
            "I": "IND",
            "S": "SUBJ",
            "M": "IMP",
            "N": "INF",
            "G": "GDIO",
            "P":"PPIO"
        },
        "tiempo": {
            "posicion": 3,
            "P":"presente",
            "I": "imperfecto",
            "F": "futuro",
            "S": "pasado",
            "C": "condicion",
            "0": "atemp"
        },
        "persona":{
            "posicion": 4
        },
        "numero":{
            "posicion": 5,
            "S":"sg",
            "P": "pl"
        }
    },
    "N":{
        "subcategoria":{ #esta subcategoría no la vamos a usar en principio porque freeling 
            # no tiene nombres propios, y puse para agregarlos aparte con otra categoría
            # PropN, pero dejo esto acá por si eventualmente queremos usarlo.
            "posicion": 1,
            "C": "COMUN", 
            "P": "PROPIO"
        },
        "genero": {
            "posicion": 2,
            "M": "masc",
            "F": "fem",
            "C": "?gen"
        },
        "numero": {
            "posicion": 3,
            "S":"sg",
            "P": "pl",
            "N": "inv"
        }
    }
}

In [3]:
class MakeRule:
    
    def __init__(self,word_dict,output="s"):
        
        self.output = output
        self.tag = word_dict.get("tag")
        self.lemma = word_dict.get("lema")
        self.shape = word_dict.get("forma")
        self.category = self.tag[0]
        self.mapping = tags_mapping.get(self.category)

        if self.mapping:
            
            self.directory = "./reglas_automaticas"
            self.file_name = f"{self.category}_rules.fcfg"
            
            for k in self.mapping.keys():
                setattr(self,k,self._get_mapped_tag(k))

            self.rule = self._make_rule()

            if self.output == "f":
                self._write_category()

            elif self.output == "s":
                print(self.rule)
                
    def _get_mapped_tag(self,rasgo):
        rasgo_mapping = self.mapping.get(rasgo)
        posicion_tag = rasgo_mapping.get("posicion")
        return rasgo_mapping.get(self.tag[posicion_tag],self.tag[posicion_tag])
                
    def _make_rule(self):
        # Habría que agregar un elif por cada categoría a la que le definamos el mapeo de keys.
        # Los atributos son las keys del diccionario. category, shape y lemma siempre se llaman
        # así porque se definen antes (es para todas las clases de palabras igual)
        # Está así cortado para que sea más legible, pero podría estar todo en una línea
        if self.category == "V":
            rule = f"{self.category}[SUBCAT={self.subcategoria},MODE={self.modo},"\
                    f"TENSE={self.tiempo},PER={self.persona},"\
                    f"NUM={self.numero},SEM=<\e.({self.lemma}(e) "\
                    f"& {self.tiempo}(e))>] -> '{self.shape}'"
        if self.category == "N":
            rule = f"{self.category}[NUM={self.numero},GEN={self.genero},"\
                    f"SEM=<\\x.({self.lemma}(x))>] -> '{self.shape}'"
        return rule
    
    def _write_category(self):
        with open(os.path.join(self.directory,self.file_name),"a+") as cat:
            cat.write(f'{self.rule}\n')
            
            

In [4]:
def create_directory(path):
    """
        Toma un path y crea la ruta. 

        Parámetros
        ----------

        path: str
            Ruta a crear.

    """
    try:
        os.makedirs(path)
        print("Se creó el directorio %s" %path)
    except OSError as e:
        if e.errno != errno.EEXIST:
            raise
            
            
# Pongo esta funcionalidad porque como el guardado va agregando reglas si lo volvés
# a correr sigue sumándolas al final.            
def remove_category_file(category):
    directory = "./reglas_automaticas"
    create_directory(directory)
    if category == "all":
        [os.remove(os.path.join(directory,f)) for f in os.listdir(directory)]
    else:
        file_name = f"{category}_rules.fcfg"
        try:
            os.remove(os.path.join(directory,file_name))
        except:
            print(f"No hay archivo {file_name} para borrar")

remove_category_file("all")
#remove_category_file("V")

In [5]:
import glob

freeling_all = list()

for freeling_file in glob.glob("./freeling/*.*.txt"):
    with open(freeling_file) as free_one:
        freeling_one = list(map(lambda a: {'forma':a[0],'lema':a[1],'tag':a[2]},
                        [f.strip("\n").split(' ') for f in free_one.readlines()]))
        freeling_all.extend(freeling_one)
        
    

# El parámetro output puede ser 's' (imprime en pantalla) 
# o 'f' (lo appendea al archivo de la clase de palabra correspondiente)

for word_index in range(len(freeling_all)):
    MakeRule(freeling_all[word_index],output="f")


In [6]:
# Agregué esto para que arme una gramática que recoja todo y poder probar la gramática (Fernando)
# Conviene probar en otra jupyter porque si no esta se sobrecarga mucho
# Para eso armé la jupyter Test

filenames = ['GramaticaDeRasgosBase.txt', 'reglas_automaticas/V_rules.fcfg']
with open('integral_grammar.fcfg', 'w') as outfile:
    for fname in filenames:
        with open(fname) as infile:
            for line in infile:
                outfile.write(line)